# Web scraping


### Code to create a dataset with Brazilian soup operas:
- website: http://teledramaturgia.com.br/
- features: TV channel, Soup opera name, date of the begining and end, director
- aim: increase data to analyse TV audience and to indicate better programs to include TV advertising

In [2]:
!pip install requests

In [2]:
!pip install bs4

  Using cached https://files.pythonhosted.org/packages/1a/b7/34eec2fe5a49718944e215fde81288eec1fa04638aa3fb57c1c6cd0f98c3/beautifulsoup4-4.8.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0b/44/0474f2207fdd601bb25787671c81076333d2c80e6f97e92790f8887cf682/soupsieve-1.9.3-py2.py3-none-any.whl


In [7]:
!pip install unidecode

  Using cached https://files.pythonhosted.org/packages/d0/42/d9edfed04228bacea2d824904cae367ee9efd05e6cce7ceaaedd0b0ad964/Unidecode-1.1.1-py2.py3-none-any.whl


In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
import re
from unidecode import unidecode

In [2]:
# open file with the names of soup operas - this will indicates the website to collect info
todas = open('titulos_2018a2019_globo.txt', 'r', encoding='UTF-8')
print(todas.read())

deus salve o rei
orgulho e paixão
segundo sol
o tempo não para
espelho da vida
o sétimo guardião
verão 90
órfãos da terra


In [11]:
# include all names into a list
with open('titulos_2018a2019_globo.txt', 'r', encoding='UTF-8') as f:
    lista = f.readlines()
    lista = [x.strip() for x in lista] 

In [12]:
lista

['deus salve o rei',
 'orgulho e paixão',
 'segundo sol',
 'o tempo não para',
 'espelho da vida',
 'o sétimo guardião',
 'verão 90',
 'órfãos da terra']

In [7]:
teste = nomes[0].replace(' ', '-')
teste

'a-favorita'

In [263]:
unidecode(teste)

'agora-e-que-sao-elas'

In [15]:
# include all corrected names into a list
novelas=[]

for i in nomes:
    cor1 = i.replace(' ', '-')
    cor2 = cor1.replace('(', '')
    cor3 = cor2.replace(')', '')
    cor4 = cor3.replace('!', '')
    cor5 = cor4.replace('...', '')
    cor6 = unidecode(cor5)
    novelas.append(cor6)

In [16]:
#lista com os nomes de todas as novelas corrigidas para entrar no link da página
print(len(novelas))
print(novelas)

8
['deus-salve-o-rei', 'orgulho-e-paixao', 'segundo-sol', 'o-tempo-nao-para', 'espelho-da-vida', 'o-setimo-guardiao', 'verao-90', 'orfaos-da-terra']


## First Test

In [17]:
#creating the lists to store the data for each soap opera 
titulo = []
horario = []
data_inicio = []
data_fim = []
autor = []

for i in novelas:
    #Request to enter the site 
    req = requests.get('http://teledramaturgia.com.br/{}/'.format(i))
    #testing whether the request was successful 
    if req.status_code == 200:
        print('Requisição bem sucedida!')
        content = req.content
    
    soup = BeautifulSoup(content, 'html.parser')
    
    #Title of the soup opera
    novela = str(soup.find_all('title'))
    novela_1 = re.findall('[-](.+?)</title>', novela)[0].lstrip()
    titulo.append(novela_1)
    
    #TV channel and soup opera schedule
    texto = str(soup.find_all('p'))
    horario_ = re.findall(', <p>.+([A-Z].+?[0-9]{2}h)<br/>', texto)
    horario.append(horario_[0])
    
    #Start and end date 
    data_ = re.findall('[de ]([0-9].+?[0-9]{4})<.+', texto)
    if len(data_) == 1:
        data_inicio.append(data_[0])
    else:
        data_inicio.append(data_[0] + ' a ' + data_[1])
        
#     data_inicio_ = re.findall('[\nde ]([0-9].+[0-9]{4})<br/>', texto)[0]
#     data_inicio.append(data_inicio_)
#     data_fim_ = re.findall('[\nde ]([0-9].+[0-9]{4})<br/>', texto)[1]
#     data_fim.append(data_fim_)
        
    #Director
    autor_ = re.findall('<p>novela de .*?([A-Z].+?)<', texto)[0]
    autor.append(autor_)
       
        

Requisição bem sucedida!
Requisição bem sucedida!
Requisição bem sucedida!
Requisição bem sucedida!
Requisição bem sucedida!
Requisição bem sucedida!
Requisição bem sucedida!
Requisição bem sucedida!


In [43]:
#example of the website text
text

'[<p>Maria da Paz (Juliana Paes) vem de uma família de justiceiros profissionais, os Ramirez, da cidade de Rio Vermelho, Espírito Santo. Desde pequena, ela mantém o hábito de ficar na cozinha ao lado de sua avó, Dulce (Fernanda Montenegro), com quem aprendeu a fazer bolos. Essa rotina a seduz mais do que as atividades impostas pelo pai, Ademir (Genézio de Barros), que quer transformá-la em uma justiceira.</p>, <p>Na década de 1990, Maria conheceu e se apaixonou por Amadeu (Marcos Palmeira), advogado formado em Vitória, porém membro do clã rival nos negócios dos Ramirez, os Matheus. Quando descobre que vem de famílias rivais, o casal, para viver o romance, sugere um pacto de paz e consegue marcar o casamento. Porém, no altar, Amadeu leva um tiro e, após a tragédia, os Matheus prometem vingança contra os Ramirez.</p>, <p>As primeiras vítimas são Fabiana (Maria Clara Baldon) e Virgínia (Duda Batista), sobrinhas de Maria, filhas de sua irmã Zenaide (Maeve Jinkings). Vicente (Álamo Facó), i

In [190]:
# example of soyp opera title before correction
titulo = str(soup.find_all('title'))
titulo

'[<title>Teledramaturgia -   A Dona do Pedaço</title>]'

In [202]:
# example of soyp opera title after correction
novela = re.findall('.+?[ -]([A-Z].+)</title>', titulo)
novela

['A Dona do Pedaço']

In [177]:
# example of schedule
horario = re.findall(', <p>.+([A-Z].+?[0-9]{2}h)<br/>', text)
horario

['Globo – 21h']

In [112]:
# example of start date
data_inicio = re.findall('(estreia:.+)</p>', text)
print(data_inicio)

['estreia: 20 de maio de 2019']


In [185]:
# example of director
autor = re.findall('[novela de].+[">]([A-Z].+)</a><br/>', text)
autor

['Walcyr Carrasco']

## Running into all soup opera's name

In [1]:
#creating the lists to store the data for each soap opera 
titulo = []
horario = []
data_inicio = []
data_fim = []
autor = []
atores = []


try:

    for i in novelas:
        #Request to enter the site 
        req = requests.get('http://teledramaturgia.com.br/{}/'.format(i))
        #testing whether the request was successful
        if req.status_code == 200:
            print('Requisição bem sucedida!')
            content = req.content

        soup = BeautifulSoup(content, 'html.parser')

        #Title of the soup opera
        novela = str(soup.find_all('title'))
        novela_1 = re.findall('[-](.+?)</title>', novela)[0].lstrip()
        titulo.append(novela_1)

        #TV channel and soup opera schedule
        texto = soup.find_all('div', class_='content-container')
        texto_2 = str(texto)

        texto_3 = str(soup.find_all('p'))

        #texto = str(soup.find_all('p'))
        horario_ = re.findall('>.+([A-Z].+?[0-9]{2}h)<br/>', texto_2)
        if len(horario_) == 0:
            horario_ = re.findall('>.+([A-Z].+?[0-9]{2}h)<br/>', texto_3)
            horario.append(horario_[0])
        else:
            horario.append(horario_[0])

        #Start and end date 
        data_ = re.findall('[de ]([0-9].+?[0-9]{4})<.+', texto_2)
        if len(data_) == 0:
            data_ = re.findall('[de ]([0-9].+?[0-9]{4})<.+', texto_3)
            if len(data_) == 1:
                data_inicio.append(data_[0])
            else:
                data_inicio.append(data_[0] + ' a ' + data_[1])
        else:
            if len(data_) == 1:
                data_inicio.append(data_[0])
            else:
                data_inicio.append(data_[0] + ' a ' + data_[1])

        #Director
        autor_ = re.findall('<p>novela de .*?([A-Z].+?)<', texto_3)[0]
        autor.append(autor_)

        #top5 Actors
        atores = re.findall('([A-Z][A-Z].+)<br/>', texto_3)
        atores_5 = atores[:5]
        for a in atores_5:
            nome = a.split()[:2]
            nome = ' '.join(nome)
            lista_a.append(nome)
        nomes_5 = ';'.join(lista_a)
        nomes_5.append(atores)

except:
    pass

In [816]:
titulo_p = titulo[0:32]
titulo_p

['O Sétimo Guardião',
 'O Tempo Não Para',
 'Órfãos da Terra',
 'Orgulho e Paixão',
 'Páginas da Vida',
 'Paraíso (1982)',
 'Paraíso Tropical',
 'Passione',
 'Pé na Jaca',
 'Pega Pega',
 'Porto dos Milagres',
 'Rock Story',
 'Sabor da Paixão',
 'Salve Jorge',
 'Sangue Bom',
 'Saramandaia (1976)',
 'Segundo Sol',
 'Senhora do Destino',
 'Sete Pecados',
 'Sete Vidas',
 'Sinhá Moça (1986)',
 'Sol Nascente',
 'Tempo de Amar',
 'Tempos Modernos',
 'Ti-ti-ti (1985)',
 'Totalmente Demais',
 'Três Irmãs',
 'Uga Uga',
 'Um Anjo Caiu do Céu',
 'Velho Chico',
 'Verão 90',
 'Verdades Secretas']

In [817]:
print(len(horario))
horario

32


['Globo – 21h',
 'Globo – 19h',
 'Globo – 18h',
 'Globo – 18h',
 'Globo – 21h',
 'Globo – 18h',
 'Globo – 21h',
 'Globo – 21h',
 'Globo – 19h',
 'Globo – 19h',
 'Globo – 20h',
 'Globo / 19h',
 'Globo – 18h',
 'Globo – 21h',
 'Globo – 19h',
 'Globo – 22h',
 'Globo – 21h',
 'Globo – 21h',
 'Globo – 19h',
 'Globo – 18h',
 'Globo – 18h',
 'Globo – 18h',
 'Globo – 18h',
 'Globo – 19h',
 'Globo – 19h',
 'Globo – 19h',
 'Globo – 19h',
 'Globo – 19h',
 'Globo – 19h',
 'Globo – 21h',
 'Globo – 19h',
 'Globo – 23h']

In [818]:
print(len(data_inicio))
data_inicio

32


['12 de novembro de 2018 a 18 de maio de 2019',
 '31 de julho de 2018 a 28 de janeiro de 2019',
 '2 de abril a 28 de setembro de 2019',
 '20 de março a 24 de setembro de 2018',
 '10 de julho de 2006 a 3 de março de 2007',
 '23 de agosto de 1982 a 26 de março de 1983',
 '5 de março a 29 de setembro de 2007',
 '17 de maio de 2010 a 15 de janeiro de 2011',
 '20 de novembro de 2006 a 16 de junho de 2007',
 '6 de junho de 2017 a 8 de janeiro de 2018',
 '5 de fevereiro a 29 de setembro de 2001',
 '9 de novembro de 2016 a 5 de junho de 2017',
 '30 de setembro de 2002 a 21 de março de 2003',
 '22 de outubro de 2012 a 18 de maio de 2013',
 '29 de abril a 2 de novembro de 2013',
 '3 de maio a 31 de dezembro de 1976',
 '14 de maio a 10 de novembro de 2018',
 '28 de junho de 2004 a 12 de março de 2005',
 '18 de junho de 2007 a 15 de fevereiro de 2008',
 '9 de março a 11 de julho de 2015',
 '28 de abril a 15 de novembro de 1986',
 '29 de agosto de 2016 a 21 de março de 2017',
 '26 de setembro de 20

In [819]:
print(len(autor))
autor

32


['Aguinaldo Silva',
 'Mário Teixeira',
 'Thelma Guedes',
 'Marcos Bernstein',
 'Manoel Carlos',
 'Benedito Ruy Barbosa',
 'Gilberto Braga',
 'Silvio de Abreu',
 'Carlos Lombardi',
 'Cláudia Souto',
 'Aguinaldo Silva',
 'Maria Helena Nascimento',
 'Ana Maria Moretzsohn',
 'Glória Perez',
 'Maria Adelaide Amaral',
 'Dias Gomes',
 'João Emanuel Carneiro',
 'Aguinaldo Silva',
 'Walcyr Carrasco',
 'Lícia Manzo',
 'Benedito Ruy Barbosa',
 'Walther Negrão',
 'Alcides Nogueira',
 'Bosco Brasil',
 'Cassiano Gabus Mendes',
 'Rosane Svartman e Paulo Halm',
 'Antônio Calmon',
 'Carlos Lombardi',
 'Antônio Calmon',
 'Benedito Ruy Barbosa',
 'Izabel de Oliveira',
 'Walcyr Carrasco']

In [820]:
#creating the dataset
d = {'Novela': titulo_p, 'Horário': horario, 'Data': data_inicio, 'Autor': autor}
df_nov2 = pd.DataFrame(data=d)
df_nov2

,Novela,Horário,Data,Autor
0,O Sétimo Guardião,Globo – 21h,12 de novembro de 2018 a 18 de maio de 2019,Aguinaldo Silva
1,O Tempo Não Para,Globo – 19h,31 de julho de 2018 a 28 de janeiro de 2019,Mário Teixeira
2,Órfãos da Terra,Globo – 18h,2 de abril a 28 de setembro de 2019,Thelma Guedes
3,Orgulho e Paixão,Globo – 18h,20 de março a 24 de setembro de 2018,Marcos Bernstein
4,Páginas da Vida,Globo – 21h,10 de julho de 2006 a 3 de março de 2007,Manoel Carlos
5,Paraíso (1982),Globo – 18h,23 de agosto de 1982 a 26 de março de 1983,Benedito Ruy Barbosa
6,Paraíso Tropical,Globo – 21h,5 de março a 29 de setembro de 2007,Gilberto Braga
7,Passione,Globo – 21h,17 de maio de 2010 a 15 de janeiro de 2011,Silvio de Abreu
8,Pé na Jaca,Globo – 19h,20 de novembro de 2006 a 16 de junho de 2007,Carlos Lombardi
9,Pega Pega,Globo – 19h,6 de junho de 2017 a 8 de janeiro de 2018,Cláudia Souto


In [ ]:
df_nov2.to_csv('novelas.csv', encoding='utf-8', sep=';')